# <font color=blue> Real Time Stock Ticker Streaming - Kafka Producer </font>


Producing the streaming data, where you can use csv modules to read and publish the data to the Kafka stream.

### Flight Data Overview:
The flight-delays and cancellation data was collected and published by the U.S. Department of Transportation’s (DOT) Bureau of Transportation Statistics. This data records the flights operated by large air carriers and tracks the on-time performance of domestic flights. This data summarises various flight information such as the number of on-time, delayed, cancelled, and diverted flights published in DOT's monthly in 2015.

In [52]:
# import required libraries
import pandas as pd
import pandas

from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import csv
from datetime import timezone
import datetime

In [53]:
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime
yf.pdr_override()

In [62]:
#stock_list = ['AMAT','LRCX','WOLF','KLAC','AAPL', 'GOOG', 'MSFT', 'AMZN']
stock_list = ['AMAT','GOOG', 'MSFT']
#stock_list = ['AMAT']

In [55]:
Share = yf.Ticker('AMAT').info
 
# Extracting the MarketPrice from the data
market_price = Share['regularMarketPrice' ]
 
# Printing the market price
print(market_price)

113.07


In [63]:

def GetHistoricalData(stock_list,numberOfYears = 0,numberOfMonths = 4):
    #print(numberOfMonths)
    end = datetime.now()
    start = datetime(end.year -  numberOfYears, end.month - numberOfMonths, end.day , end.hour, end.minute,end.second)
    
    historicalData = []
    for stock in stock_list:
        #print(start,end)
        df = yf.download(stock, start, end)
        df = df.reset_index()
        df['Label'] = stock
        df['Date'] = pd.to_datetime(df['Date'])
        historicalData.append(df)
    return(historicalData)

In [64]:
def GetLiveData(symbol):
    ticker = yf.Ticker(symbol).info
    market_price = ticker['regularMarketPrice']
    market_open = ticker['regularMarketOpen']
    market_High = ticker['regularMarketDayHigh']
    market_Low = ticker['regularMarketDayLow']
    market_volume = ticker['regularMarketVolume']
    market_symbol = ticker['symbol']
    #print('Ticker: AMAT')
    #print('Market Price:', market_price)
    #print('Previous Close Price:', previous_close_price)
    liveData = {'Date':[pd.to_datetime(datetime.now())],'Open': [market_open] , 'High':[market_High],'Low':[market_Low],'Close':[market_price],'Adj Close':[market_price],'Volume':[market_volume],'Label':[market_symbol]}
    return(pd.DataFrame(liveData))

In [65]:
historicalData = GetHistoricalData(stock_list,1,0)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [70]:
historicalData[0]['Close'][len(historicalData[0]['Close'])-1]

113.38

In [69]:
liveData = GetLiveData('AMAT')
historicalData[0] = pandas.concat([historicalData[0],liveData], ignore_index=True)

In [ ]:
liveData.head()

In [ ]:
historicalData = GetHistoricalData(1,0)

for stock in range(len(historicalData)):
    symbol = historicalData[stock]['Label'][0]
    liveData = GetLiveData(symbol)
    liveData['Date'] = pd.to_datetime(liveData['Date'])
    historicalData[stock] = pandas.concat([historicalData[stock],liveData], ignore_index=True)
    historicalData[stock]['Date'] =historicalData[stock]['Date'].astype(str)

In [ ]:
all_data = []
for stock in range(len(historicalData)):
    all_data.append(historicalData[stock].to_dict(orient='records'))


In [ ]:
topic = 'stock_ticker'
publish_message(producer, topic, all_data)

In [ ]:
df_all = df_Historical.append(df_live, ignore_index = True)


In [ ]:
df_all.head()

In [ ]:
df_all.tail()

In [ ]:
df_all['Date'] =df_all['Date'].astype(str)

In [ ]:
all_data = df_all.to_dict(orient='records')
print(all_data[0])

In [ ]:
len(all_data)

## Step 3

#### Create publisher and producer function

Functions are taken from the Week 9 Lab material provided.

In [59]:
# function that publishes the message 
def publish_message(producer_instance, topic_name, data):
    #print(data)
    try:
        producer_instance.send(topic_name, data)
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

        
# function that connects the kafka producer        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['192.168.86.48:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
 

In [60]:
producer = connect_kafka_producer()

## Step 4

#### Send desired data batches 

Using the above created functions we create the desired flight data batches and send them so the consumers can receive the data.

In [71]:
if __name__ == '__main__':
    try:
   
    
        ## SET TOPIC AND DATA TO BE SENT

        topic = 'stock_ticker'

        #all_data = df.to_dict(orient='records')

        ## SET THE PRODUCERS

        #producer = connect_kafka_producer()



        ## GET DATA AND META DATA FOR EACH KEY


        iteration_counter = 0

        #stock_list = ['AMAT','LRCX','WOLF','KLAC','AAPL', 'GOOG', 'MSFT', 'AMZN']
        stock_list = ['AMAT','GOOG', 'MSFT']
        #stock_list = ['AMAT']
        #stock_list = ['AMAT','LRCX','WOLF','KLAC']
        # start the data publishing process
        #print('Publishing records for ', stock_list, '..')

        # set a continous loop to produce and publish data
        
        #print(len(df_Historical))
        month = 3
        df_Historical = GetHistoricalData(stock_list,1,0)
        while True:
            """
            if month == 0:
                month = 3
            df_Historical = GetHistoricalData(stock_list,1,month)
            
            month = month - 1
            """
            for stock in range(len(df_Historical)):
                    symbol = df_Historical[stock]['Label'][0]
                    liveData = GetLiveData(symbol)
                    liveData['Date'] = pd.to_datetime(liveData['Date'])
                    df_Historical[stock] = pandas.concat([df_Historical[stock],liveData], ignore_index=True)
                    df_Historical[stock]['Date'] = df_Historical[stock]['Date'].astype(str)
                    sleep(20)

            all_data = []
            for stock in range(len(df_Historical)):
                all_data.append(df_Historical[stock].to_dict(orient='records'))


            publish_message(producer, topic, all_data)



            if iteration_counter > 1000:
                    break

            iteration_counter += 1   

            # send producer to sleep 
            sleep(120)
        print("Exited with ", iteration_counter, " iterations")
    except Exception as ex:
        print("exception after ", iteration_counter, " iterations")
        print(str(ex))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
exception after  44  iterations
HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Read timed out. (read timeout=30)
